In [1]:
from google.colab import drive
drive.mount('/content/drive')  #listになかったのでドライブ上にデータ保存し、ドライブをマウント。

Mounted at /content/drive


In [2]:
! pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.1 MB/s eta 0:00:00


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/signate/モジュール系/EDA')
sys.path.append('/content/drive/MyDrive/signate/モジュール系/Feature Engineering')
sys.path.append('/content/drive/MyDrive/signate/モジュール系/meanF1')

In [4]:
### 自作モジュール
from plot_qq_plots import plot_qq_plots
from reduce_mem import reduce_mem_usage
from show_corr import show_corr
from target_encoded import get_kfold, get_targetencoding, to_target
from obj_to_cat import to_cat_data
from normalize_col import normalize_columns
from combined_features import create_combined_features
from threshold_tuning_binary import threshold_tuning_binary

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold

from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.preprocessing import StandardScaler

In [6]:
feat = "feat00"
train = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_train_{feat}.csv")
test = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_test_{feat}.csv")
train_origin = pd.read_csv("/content/drive/MyDrive/signate/債務不履行/データセット/train.csv")

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

# 説明変数のカラム
cols_exp = [c for c in test.columns]

###-----------------------------------------------------------------------------
## カテゴリ変数の抽出
cols_notcat = ['MIS_Status', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
               'DisbursementGross', 'GrAppv', 'SBA_Appv','population', 'density',
               'lr_avg_ppi','ApprovalToDisbursement_days','outlier'] # 数値特徴量 'DisbursementGross_minus_SBA_Appv'
# county_col = ['county_gdp','sector_count']
time_col = ["DisbursementDate_timestamp",	"ApprovalDate_timestamp",	"LoanEnd_timestamp"]
dissba = ['DisbursementGross_plus_SBA_Appv','DisbursementGross_minus_SBA_Appv', 'DisbursementGross_times_SBA_Appv','DisbursementGross_divided_by_SBA_Appv']
sbagr = ['SBA_Appv_plus_GrAppv','SBA_Appv_minus_GrAppv', 'SBA_Appv_times_GrAppv','SBA_Appv_divided_by_GrAppv']
disgr = ['DisbursementGross_plus_GrAppv','DisbursementGross_minus_GrAppv', 'DisbursementGross_times_GrAppv','DisbursementGross_divided_by_GrAppv']
# year_col = ["USREC_SUM"]
# urb_col = ["HOUPCT_RUR","ALAND_PCT_RUR","RUR_pct"]
city_col = ["city_Term_mean","city_NoEmp_mean"]

cols_notcat = cols_notcat + city_col + dissba + sbagr + disgr + time_col # 数値特徴量
###-----------------------------------------------------------------------------

col_cat = [c for c in train.columns if not c in cols_notcat] # カテゴリ特徴量

# ターゲットエンコカラム
target_encoded_cols = [c for c in train.columns if "target_" in c]
# ordinalカテゴリ
ordinal_cols_cat = [c for c in col_cat if not c in target_encoded_cols]

# カテゴリ特徴量のカラム
cols_cat = ordinal_cols_cat + target_encoded_cols

for col in cols_cat:
    # もし欠損値があればその特徴量のユニーク数(欠損値を除く)で埋める
    num_null = train[col].nunique() - 1
    train[col].fillna(num_null, inplace=True)
    test[col].fillna(num_null, inplace=True)

# 100以上のユニーク数を持つ特徴量を削除
cols_to_remove = [c for c in cols_exp if train[c].nunique() > 100]
cols_exp = [c for c in cols_exp if c not in cols_to_remove]
cols_cat = [c for c in cols_cat if c not in cols_to_remove]

Memory usage of dataframe is 13.23 MB
Memory usage after optimization is: 2.95 MB
Decreased by 77.7%
Memory usage of dataframe is 12.91 MB
Memory usage after optimization is: 2.91 MB
Decreased by 77.5%


In [7]:
train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,MIS_Status,Sector,...,DisbursementGross_plus_GrAppv,DisbursementGross_minus_GrAppv,DisbursementGross_times_GrAppv,DisbursementGross_divided_by_GrAppv,lr_avg_ppi,outlier,DisbursementDate_timestamp,ApprovalDate_timestamp,LoanEnd_timestamp,ApprovalToDisbursement_days
0,163,21,1.0,0,0,1,1,3,1,23,...,-0.392578,-0.09137,-0.249634,-0.248779,0.435791,0.090149,-0.559570,0.846680,0.112244,0.353271
1,84,6,1.0,4,0,0,0,3,1,19,...,0.448242,-0.09137,-0.025345,-0.248779,-0.380371,0.113342,-1.242188,-1.584961,-1.061523,0.496338
2,242,45,1.0,4,90,0,1,3,1,7,...,-0.591797,-0.05780,-0.265625,-0.129028,0.137207,0.020538,0.015060,-0.081238,1.213867,0.482666
3,237,4,1.0,0,0,0,1,3,1,6,...,0.212524,-0.09137,-0.113708,-0.248779,-0.122314,0.114258,0.978027,0.340332,1.818359,0.533203
4,184,0,1.0,0,0,0,1,3,1,23,...,1.415039,-0.09137,0.544922,-0.248779,-0.623535,0.062927,-2.912109,-0.309570,-1.285156,0.240234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,283,14,1.0,0,0,1,1,3,1,23,...,-0.392578,-0.09137,-0.249634,-0.248779,0.009750,0.087708,-0.559570,-1.128906,1.195312,0.519043
42303,53,2,1.0,0,0,0,4,3,1,7,...,-0.697266,-0.09137,-0.268311,-0.248779,-0.444092,0.063965,-1.656250,0.966797,-1.620117,0.245239
42304,59,6,2.0,0,0,1,1,3,1,7,...,-0.473877,-0.09137,-0.257812,-0.248779,1.329102,0.105591,0.254883,0.243896,-0.276611,0.476807
42305,295,18,1.0,0,8,0,1,3,1,7,...,0.476562,-0.09137,-0.013344,-0.248779,0.176514,0.024857,-0.582520,-2.072266,1.289062,0.596191


**Tabnet**

In [8]:
def train_tabnet(train, cols_exp, cols_cat, col_target, params=None):
    if params is None:
        params = {}

    params_add = {"device_name": "cpu", "seed": 0}
    params |= params_add

    x = train[cols_exp].values
    y = train[col_target].values

    # カテゴリ変数のインデックスと次元
    cols_cat_idxs = [cols_exp.index(c) for c in cols_cat]
    cols_num_idxs = [cols_exp.index(c) for c in cols_exp if c not in cols_cat]
    cols_cat_dims = [train[c].nunique() for c in cols_cat]

    # K-fold
    kf = KFold(n_splits=4, shuffle=True, random_state=1)
    y_valid_pred_lst = []
    idx_valid_lst = []
    clf_lst = []

    for fold, (idx_train, idx_valid) in enumerate(kf.split(x)):
        print("fold", fold)
        x_train, x_valid = x[idx_train], x[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        # 数値特徴量の正規化
        if cols_num_idxs:  # cols_num_idxsが空でない場合にのみ実行
            scaler = StandardScaler()
            # 数値特徴量にのみスケーラーを適用
            x_train[:, cols_num_idxs] = scaler.fit_transform(x_train[:, cols_num_idxs])
            x_valid[:, cols_num_idxs] = scaler.transform(x_valid[:, cols_num_idxs])

        # モデリング
        pretrainer = TabNetPretrainer(**params)
        pretrainer.fit(x_train, eval_set=[x_valid])
        clf = TabNetClassifier(**params, cat_idxs=cols_cat_idxs, cat_dims=cols_cat_dims)
        clf.fit(
            x_train, y_train,
            eval_set=[(x_train, y_train), (x_valid, y_valid)],
            eval_name=['train', 'valid'],
            eval_metric=["logloss"],
            from_unsupervised=pretrainer
        )

        # OOF予測
        y_valid_pred = clf.predict_proba(x_valid)
        y_valid_pred_lst.append(y_valid_pred)
        idx_valid_lst.append(idx_valid)
        clf_lst.append(clf)

    # OOF予測の統合
    idx_valid = np.hstack(idx_valid_lst)
    y_valid_pred = np.vstack(y_valid_pred_lst)
    oof_pred = y_valid_pred[np.argsort(idx_valid)]

    return clf_lst, oof_pred

In [9]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [10]:
col_target = "MIS_Status"

# train tabnet
clf_lst, oof_pred = train_tabnet(train, cols_exp, cols_cat, col_target)

# normalization for test
x_train = train[cols_exp].to_numpy()
x_test = test[cols_exp].to_numpy()
cols_num_idxs = [i for i, c in enumerate(cols_exp) if not c in cols_cat]

# cols_num_idxsが空でない場合にのみ実行
if cols_num_idxs:  # cols_num_idxsが空でない場合にのみ実行
  scaler = StandardScaler()
  scaler.fit(x_train[:, cols_num_idxs])
  x_test[:, cols_num_idxs] = scaler.transform(x_test[:, cols_num_idxs])

# predict test with CV ensemble
y_test_pred = predict_test(x_test, clf_lst)

# record
oof_pred_df = pd.DataFrame(oof_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
test_pred_df = pd.DataFrame(y_test_pred, columns=[f"MIS_Status_{h}" for h in range(2)])

fold 0


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.56977 | val_0_unsup_loss_numpy: 2.684619903564453|  0:00:04s
epoch 1  | loss: 1.37578 | val_0_unsup_loss_numpy: 1.1567000150680542|  0:00:10s
epoch 2  | loss: 1.01279 | val_0_unsup_loss_numpy: 1.075600028038025|  0:00:14s
epoch 3  | loss: 0.98907 | val_0_unsup_loss_numpy: 1.085919976234436|  0:00:16s
epoch 4  | loss: 0.98647 | val_0_unsup_loss_numpy: 1.0653899908065796|  0:00:18s
epoch 5  | loss: 0.97298 | val_0_unsup_loss_numpy: 1.0187499523162842|  0:00:21s
epoch 6  | loss: 0.969   | val_0_unsup_loss_numpy: 0.9766299724578857|  0:00:23s
epoch 7  | loss: 0.96391 | val_0_unsup_loss_numpy: 0.964900016784668|  0:00:25s
epoch 8  | loss: 0.96622 | val_0_unsup_loss_numpy: 0.9339100122451782|  0:00:27s
epoch 9  | loss: 0.96587 | val_0_unsup_loss_numpy: 0.9185799956321716|  0:00:29s
epoch 10 | loss: 0.96219 | val_0_unsup_loss_numpy: 0.8606299757957458|  0:00:31s
epoch 11 | loss: 0.94687 | val_0_unsup_loss_numpy: 0.8760700225830078|  0:00:34s
epoch 12 | loss: 0.94565 | val_0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [5, 7, 24, 38, 51, 52, 3] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [1, 1, 1, 1, 1, 1, 1] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [3, 4, 5, 6, 7, 8, 9] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining

epoch 0  | loss: 0.45892 | train_logloss: 0.35555 | valid_logloss: 0.35777 |  0:00:02s
epoch 1  | loss: 0.3119  | train_logloss: 0.40839 | valid_logloss: 0.40723 |  0:00:05s
epoch 2  | loss: 0.30676 | train_logloss: 0.40498 | valid_logloss: 0.40285 |  0:00:07s
epoch 3  | loss: 0.30502 | train_logloss: 0.33037 | valid_logloss: 0.33233 |  0:00:09s
epoch 4  | loss: 0.30352 | train_logloss: 0.30812 | valid_logloss: 0.31104 |  0:00:13s
epoch 5  | loss: 0.30021 | train_logloss: 0.30413 | valid_logloss: 0.30838 |  0:00:15s
epoch 6  | loss: 0.3014  | train_logloss: 0.30252 | valid_logloss: 0.30945 |  0:00:18s
epoch 7  | loss: 0.30192 | train_logloss: 0.29684 | valid_logloss: 0.30416 |  0:00:20s
epoch 8  | loss: 0.29828 | train_logloss: 0.29689 | valid_logloss: 0.3016  |  0:00:22s
epoch 9  | loss: 0.29978 | train_logloss: 0.29628 | valid_logloss: 0.30145 |  0:00:26s
epoch 10 | loss: 0.29681 | train_logloss: 0.29599 | valid_logloss: 0.30191 |  0:00:28s
epoch 11 | loss: 0.29713 | train_logloss: 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 3.664   | val_0_unsup_loss_numpy: 2.549499988555908|  0:00:01s
epoch 1  | loss: 1.41366 | val_0_unsup_loss_numpy: 1.1893500089645386|  0:00:03s
epoch 2  | loss: 0.99669 | val_0_unsup_loss_numpy: 1.0238399505615234|  0:00:06s
epoch 3  | loss: 0.98819 | val_0_unsup_loss_numpy: 0.9657800197601318|  0:00:09s
epoch 4  | loss: 0.97443 | val_0_unsup_loss_numpy: 0.9454799890518188|  0:00:10s
epoch 5  | loss: 0.97241 | val_0_unsup_loss_numpy: 0.9264000058174133|  0:00:12s
epoch 6  | loss: 0.96694 | val_0_unsup_loss_numpy: 0.9080399870872498|  0:00:14s
epoch 7  | loss: 0.96195 | val_0_unsup_loss_numpy: 0.8921399712562561|  0:00:16s
epoch 8  | loss: 0.95106 | val_0_unsup_loss_numpy: 0.8582500219345093|  0:00:19s
epoch 9  | loss: 0.94926 | val_0_unsup_loss_numpy: 0.8512099981307983|  0:00:22s
epoch 10 | loss: 0.94057 | val_0_unsup_loss_numpy: 0.8219699859619141|  0:00:23s
epoch 11 | loss: 0.93551 | val_0_unsup_loss_numpy: 0.8164399862289429|  0:00:25s
epoch 12 | loss: 0.93681 | va

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [5, 7, 24, 38, 51, 52, 3] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [1, 1, 1, 1, 1, 1, 1] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [3, 4, 5, 6, 7, 8, 9] to []
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining

epoch 0  | loss: 0.53279 | train_logloss: 0.41406 | valid_logloss: 0.41376 |  0:00:02s
epoch 1  | loss: 0.32755 | train_logloss: 0.3997  | valid_logloss: 0.39972 |  0:00:04s
epoch 2  | loss: 0.31809 | train_logloss: 0.36013 | valid_logloss: 0.36132 |  0:00:08s
epoch 3  | loss: 0.31273 | train_logloss: 0.34692 | valid_logloss: 0.34593 |  0:00:10s
epoch 4  | loss: 0.30509 | train_logloss: 0.32104 | valid_logloss: 0.32193 |  0:00:12s
epoch 5  | loss: 0.30358 | train_logloss: 0.31639 | valid_logloss: 0.31666 |  0:00:15s
epoch 6  | loss: 0.30286 | train_logloss: 0.30305 | valid_logloss: 0.30275 |  0:00:17s
epoch 7  | loss: 0.30269 | train_logloss: 0.3002  | valid_logloss: 0.299   |  0:00:21s
epoch 8  | loss: 0.30058 | train_logloss: 0.2965  | valid_logloss: 0.2951  |  0:00:23s
epoch 9  | loss: 0.29901 | train_logloss: 0.29963 | valid_logloss: 0.29905 |  0:00:25s
epoch 10 | loss: 0.29895 | train_logloss: 0.29559 | valid_logloss: 0.29497 |  0:00:28s
epoch 11 | loss: 0.29883 | train_logloss: 0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [11]:
from sklearn.metrics import f1_score

# 実際のターゲット値
y_true = train[col_target].values

# OOF予測値をクラスに変換（確率から最も高いクラスを選択）
# 二値分類の場合、0.5を閾値としてクラスを決定することが一般的です
y_oof_pred_class = (oof_pred[:, 1] > 0.5).astype(int)

# mean F1スコア（マクロ平均）を計算
mean_f1_score_macro = f1_score(y_true, y_oof_pred_class, average='macro')
print(f'Mean F1 Score (Macro): {mean_f1_score_macro}')


Mean F1 Score (Macro): 0.62419393785348


In [12]:
# save
oof_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_tabnet_{feat}.csv", index=False)
test_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_tabnet_{feat}.csv", index=False)